

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`. 

In [2]:
for data, target in ds:
    print(data, data.size())
    print(target, target.size())
    break

tensor([ 1, 17, 23,  3,  5, 24, 24, 22, 20, 26,  8, 21,  4, 11,  2]) torch.Size([15])
tensor([ 0,  0,  0,  0,  1, 17, 23,  3,  5, 24, 24, 22, 20, 26,  8]) torch.Size([15])


In [3]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.hidden_size = hidden_size
    self.gru = torch.nn.GRU(
        input_size=N+1, 
        hidden_size=hidden_size, 
        num_layers=1,
        batch_first=True
    )
    
    self.decoder = torch.nn.Linear(
        in_features=hidden_size,
        out_features=N+1
    )
    self.hidden = None
    
    
  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass

    x, _ = self.gru(x)
    logits = self.decoder(x)
    return logits
    pass

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z. 
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    s = torch.tensor([ord(char)-ord('a')+1 for char in s])
    one_hot = idx_to_onehot(s).unsqueeze(0).to(device)
    
    logits = self.forward(one_hot)
    _, pred = logits.max(dim=-1)
    result = ''.join([chr(idx.item()+ord('a')-1) if idx.item()>0 else ' ' for idx in pred[0]])
    return result


## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [4]:
D = DELAY
def test_model(model, sequence_length=15):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
    result = model.test_run(s)
#     print(f'target: {s}, result: {result}')
    for c1, c2 in zip(s[:-D], result[D:]):
      correct += int(c1 == c2)
    total += len(s) - D

  return correct / total

In [5]:
from tqdm import tqdm
def train():
    
    batch_size = 32
    train_dataloader = torch.utils.data.DataLoader(ds, batch_size=batch_size)
    
    model.train()

    correct = 0
    total = 0
    
    for data, target in tqdm(train_dataloader):
        optimizer.zero_grad()
        x = idx_to_onehot(data)
        t = idx_to_onehot(target)[:, DELAY:]
#         t = idx_to_onehot(target)
        x, t = x.to(device), t.to(device)
    
        logits = model(x)[:, DELAY:, :]
#         logits = model(x)
        loss = criterion(logits, t)
        loss.backward()
        optimizer.step()
    scheduler.step()
    
    accuracy = test_model(model)
    return accuracy, loss.item()
        
        

In [6]:
import time
start_time = time.time()

h_units = 128

# TODO
if torch.cuda.is_available():
  device = torch.device("cuda:0")
elif torch.backends.mps.is_available():
  device = torch.device("mps")
else:
  device = torch.device("cpu")

model = GRUMemory(hidden_size=h_units).to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.8)

model = model.to(device)
criterion = criterion.to(device)

n_epochs = 5
for epoch in range(1, n_epochs+1):
    accuracy, loss = train()
    print(f'epoch: {epoch}, accuracy: {accuracy}, loss: {loss}')

# model = model.to('cpu')
model.eval()

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

100%|██████████| 6250/6250 [00:18<00:00, 346.49it/s]


epoch: 1, accuracy: 1.0, loss: 0.117104671895504


100%|██████████| 6250/6250 [00:16<00:00, 368.42it/s]


epoch: 2, accuracy: 0.9998750156230471, loss: 0.09992551058530807


100%|██████████| 6250/6250 [00:16<00:00, 368.49it/s]


epoch: 3, accuracy: 1.0, loss: 0.10654208809137344


100%|██████████| 6250/6250 [00:16<00:00, 368.83it/s]


epoch: 4, accuracy: 0.997531473710195, loss: 0.08404254913330078


100%|██████████| 6250/6250 [00:16<00:00, 368.03it/s]


epoch: 5, accuracy: 0.9998747181157605, loss: 0.09709060192108154
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [7]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size
  
  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [14]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.max_delay = max_delay
    
    self.gru = torch.nn.GRU(
        input_size=N+1+max_delay+1, #a new dimension for delay
        hidden_size=hidden_size, 
        num_layers=2,
        batch_first=True
    )
    
    self.decoder = torch.nn.Linear(
        in_features=hidden_size,
        out_features=N+1
    )

  def forward(self, x, delays):
    # inputs:
    # x - tensor of shape (batch size, seq length, N + 1)
    # delays - tensor of shape (batch size)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    
    d_onehot = idx_to_onehot(delays, k=self.max_delay+1)
    d_onehot = d_onehot.to(device)
    
    batch_size, seq_length, _ = x.size()
    
    x = torch.cat((x, d_onehot.unsqueeze(1).repeat(1, seq_length, 1)), dim=-1)
#     print(x.size())

    x, _ = self.gru(x)
    logits = self.decoder(x)
    
    return logits
    # TODO
    pass

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z, 
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings, 
    # then get the result from your network, and then convert the output 
    # back to a string of the same length, with 0 mapped to ' ', 
    # and 1-26 mapped to a-z.

    # TODO
    s = torch.tensor([ord(char)-ord('a')+1 for char in s])
    one_hot = idx_to_onehot(s).unsqueeze(0).to(device)
    
    logits = self.forward(one_hot, torch.tensor(delay).unsqueeze(0))
    _, pred = logits.max(dim=-1)
    result = ''.join([chr(idx.item()+ord('a')-1) if idx.item()>0 else ' ' for idx in pred[0]])
    return result
    
    pass


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [15]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [16]:
def train():
    
    batch_size = 32
    train_dataloader = torch.utils.data.DataLoader(ds_delay, batch_size=batch_size)
    
    model.train()
    
    for data, delay, target in tqdm(train_dataloader):
        optimizer.zero_grad()
        x = idx_to_onehot(data)
        t = idx_to_onehot(target)
        x, t = x.to(device), t.to(device)
    
        logits = model(x, delay)
        loss = criterion(logits, t)
        loss.backward()
        optimizer.step()
    scheduler.step()
    
    accuracy = test_variable_delay_model(model)
    return accuracy, loss.item()

In [20]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20
DATASET_SIZE = 200000

ds_delay = VariableDelayEchoDataset(MAX_DELAY, SEQ_LENGTH, DATASET_SIZE)

# TODO: implement model training here.
model = VariableDelayGRUMemory(
    hidden_size=256,
    max_delay = MAX_DELAY
)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=1, gamma=0.2)

model = model.to(device)
criterion = criterion.to(device)

test_variable_delay_model(model)

n_epochs = 5
for epoch in range(1, n_epochs+1):
    accuracy, loss = train()
    print(f'epoch: {epoch}, accuracy: {accuracy}, loss: {loss}')

# model = model.to('cpu')
model.eval()


end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

100%|██████████| 6250/6250 [00:23<00:00, 264.66it/s]


epoch: 1, accuracy: 0.9765084075173096, loss: 0.48666226863861084


100%|██████████| 6250/6250 [00:23<00:00, 264.56it/s]


epoch: 2, accuracy: 0.9901163518078319, loss: 0.43324175477027893


100%|██████████| 6250/6250 [00:23<00:00, 264.76it/s]


epoch: 3, accuracy: 0.9925655241935484, loss: 0.5323887467384338


100%|██████████| 6250/6250 [00:23<00:00, 264.77it/s]


epoch: 4, accuracy: 0.9937911337389793, loss: 0.5303991436958313


100%|██████████| 6250/6250 [00:23<00:00, 264.81it/s]


epoch: 5, accuracy: 0.9924430128840436, loss: 0.477691650390625
tests passed
